## Train a model locally or remote

In [1]:
%cd ~/SageMaker/ezsm-ray-FM/dev/

/home/ec2-user/SageMaker/ezsm-ray-FM/dev


In [2]:
%pip uninstall -y ezsmdeploy
%pip install -e ../

Found existing installation: ezsmdeploy 2.0.dev1
Uninstalling ezsmdeploy-2.0.dev1:
  Successfully uninstalled ezsmdeploy-2.0.dev1
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Obtaining file:///home/ec2-user/SageMaker/ezsm-ray-FM
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ezsmdeploy
Note: you may need to restart the kernel to use updated packages.


### Note: you may need to restart the kernel to use updated packages.

## Testing foundation model 

In [3]:
# %pip uninstall -y sagemaker
# %pip install --upgrade sagemaker==2.148.0 --no-cache-dir

In [4]:
import sagemaker
sagemaker.__version__

'2.148.0'

In [5]:
!pip show ezsmdeploy

Name: ezsmdeploy
Version: 2.0.dev1
Summary: SageMaker custom deployments made easy
Home-page: https://pypi.python.org/pypi/ezsmdeploy
Author: Shreyas Subramanian
Author-email: subshrey@amazon.com
License: MIT
Location: /home/ec2-user/SageMaker/ezsm-ray-FM
Requires: boto3, sagemaker, sagemaker-studio-image-build, shortuuid, yaspin
Required-by: 


In [6]:
import ezsmdeploy

In [7]:
ezsmdeploy.utils().list_foundation_models()

List of foundation models in Jumpstart: 

huggingface-text2text-bart4csc-base-chinese
huggingface-text2text-bigscience-t0pp
huggingface-text2text-bigscience-t0pp-bnb-int8
huggingface-text2text-bigscience-t0pp-fp16
huggingface-text2text-flan-t5-base
huggingface-text2text-flan-t5-base-samsum
huggingface-text2text-flan-t5-large
huggingface-text2text-flan-t5-small
huggingface-text2text-flan-t5-xl
huggingface-text2text-flan-t5-xxl
huggingface-text2text-flan-t5-xxl-bnb-int8
huggingface-text2text-flan-t5-xxl-fp16
huggingface-text2text-flan-ul2-bf16
huggingface-text2text-pegasus-paraphrase
huggingface-text2text-qcpg-sentences
huggingface-text2text-t5-one-line-summary


In [8]:
ezonsm = ezsmdeploy.Deploy(model = "huggingface-text2text-flan-ul2-bf16",
                           foundation_model=True)

0:00:00.231519 | created model(s). Now deploying on ml.g5.12xlarge
0:10:04.229508 | deployed model-!
0:10:04.229575 | Done! ✔ 


In [9]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [10]:
ezonsm.predictor.serializer = JSONSerializer()
ezonsm.predictor.deserializer = JSONDeserializer()

### Ezsmdeploy chat testing interface

In [11]:
# ezonsm.chat()

### Indvidual Utterance testing

In [12]:
payload = {
    "text_inputs": "<human>: Tell me the steps to make a pizza\n<bot>:",
    "max_length": 500,
    "max_time": 50,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
    "stopping_criteria": ["<human>"],
}
response = ezonsm.predictor.predict(payload)
print(response[0][0]["generated_text"])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027str\u0027 object is not callable"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/ezsm-foundation-endpoint-mt33lmjzdzs7mkdue7qijp in account 607789152815 for more information.

In [ ]:
!git clone --branch v0.16 --depth 1 https://github.com/togethercomputer/OpenChatKit.git

## Deploy on SageMaker

In [ ]:
# ezonsm = ezsmdeploy.Deploy(model = 'model.joblib', # if you intend to add models later, pass model as list, otherwise str
#                   script = 'modelscript_sklearn.py',
#                   requirements = ['scikit-learn==1.2.1','numpy==1.22.0','joblib==1.2.0'], #or pass in the path to requirements.txt
#                   autoscale = True, 
#                   wait = True)

To debug docker build errors try this....

In [ ]:
# !./src/build-docker.sh test

In [ ]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"
import boto3, json

def query_endpoint(encoded_text, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/x-text", Body=encoded_text
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_text"]
    return generated_text

In [ ]:
# out = ezonsm.predictor.predict(digits.data[-1:].tobytes())#.decode()
# out

In [ ]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

text1 = "Translate to German:  My name is Arthur"
text2 = "A step by step recipe to make bolognese pasta:"


for text in [text1, text2]:
    query_response = query_endpoint(text.encode("utf-8"), endpoint_name="ezsmdeploy-endpoint-gve5ccdmw3riwtggzqx4bw")
    generated_text = parse_response(query_response)
    print(
        f"Inference:{newline}"
        f"input text: {text}{newline}"
        f"generated text: {bold}{generated_text}{unbold}{newline}"
    )

In [ ]:
# ezonsm.predictor.predict

### Don't leave resources running

In [ ]:
ezonsm.predictor.delete_endpoint()